In [1]:
import os,glob,zipfile
from Code import HBV_pso
from Code import nam_pso
from Code import GR4J_pso
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display,  clear_output
from ipysheet import from_array, to_array
import ipysheet
from ipywidgets import FileUpload


/home/cak/Desktop/jupyter/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Model Selection 

In [2]:
files = glob.glob1('./Data','*.csv')

file = widgets.Dropdown(
    options=files,
    description='Data:',
    disabled=False,
)

w = widgets.Dropdown(
    options=['NAM', 'HBV', 'GR4J'],
    value='HBV',
    description='Model:',
    disabled=False,
)


area = widgets.BoundedFloatText(
    value=100,
    min=0,
    max=5000.0,
    step=0.1,
    description='Area:',
    disabled=False
)

cal = widgets.RadioButtons(
    options=['True','False'],
    value='False',
    description='Calibration <br>: ',
    disabled=False
)

m = widgets.Dropdown(
    options=['SLSQP', 'PSO', 'BFGS','Newton-CG'],
    value='SLSQP',
    description='Method:',
    disabled=False,
)

max_ = widgets.IntText(
    value=7,
    description='Max Iteration:',
    disabled=False
)


spin = widgets.IntText(
    value=7,
    description='Spinoff Day:',
    disabled=False
)


obj = widgets.Dropdown(
    options=['NSE', 'KGE', 'RMSE','R2','RMPW','NSLF'],
    value='NSE',
    description='Objective',
    disabled=False,
)



display(file,w,area,cal,m,max_,obj)

Dropdown(description='Data:', options=('Cakit.csv', 'L0123002_1984_2012.csv', 'L0123001_2005_2012.csv', 'L0123…

Dropdown(description='Model:', index=1, options=('NAM', 'HBV', 'GR4J'), value='HBV')

BoundedFloatText(value=100.0, description='Area:', max=5000.0, step=0.1)

RadioButtons(description='Calibration <br>: ', index=1, options=('True', 'False'), value='False')

Dropdown(description='Method:', options=('SLSQP', 'PSO', 'BFGS', 'Newton-CG'), value='SLSQP')

IntText(value=7, description='Max Iteration:')

Dropdown(description='Objective', options=('NSE', 'KGE', 'RMSE', 'R2', 'RMPW', 'NSLF'), value='NSE')

In [6]:
if w.value == 'HBV':
    params = [1.06653896e+00, 7.85803777e-01, 1.12456396e+00, 4.00000005e-02
        , 6.52779758e+01, 1.40000000e+00, 2.04563613e-01, 5.00000000e-01
        , 7.23806920e-03, 6.20000000e-04, 5.98308241e-01, 2.29012429e+00
        , 2.32816764e-02, 4.01249854e-01, 0.00000000e+00, 1.25000000e-01
        , 1.19040117e+00, 1.40000000e+00, 6.15462361e+00]
    States = np.array([0, 0, 0.9 * params[1], 0, 0, 0, 0, 0.1])
elif w.value == 'NAM':
        params = [6.96780205e+00, 4.86098809e+02, 6.66247792e-01, 5.42601108e+02
        , 2.43815545e+01, 8.21285865e-01, 1.00000000e-02, 1.00000000e-02
        , 7.37979357e+02, 9.64180895e-01, 2.06295770e+00]
        States = np.array([0, 0, 0.9 * params[1], 0, 0, 0, 0, 0.1])
else:
        params = np.array([350, 0.5, 90, 0.5, 0.5, 4])

In [7]:
print(w.value,file.value,area.value,cal.value,m.value,max_.value,obj.value)

NAM Darbogaz.csv 100.0 True SLSQP 7 NSE


In [8]:
button = widgets.Button(description="Run")
output = widgets.Output()
display(button,output)


def on_button_clicked(b):
    with output:
        if w.value == 'HBV':
            n = HBV_pso.Hbv(area.value, params, States, calibration=eval(cal.value), method=m.value, Objective_fun=obj.value.lower(), maxiter=max_.value)
        elif w.value == 'NAM':
            n = nam_pso.Nam(area.value, params,States, calibration=eval(cal.value), method=m.value, Objective_fun=obj.value.lower(),maxiter=max_.value)
        else:
            n = GR4J_pso.GR4J(area.value, params, params, calibration=eval(cal.value), method=m.value, Objective_fun=obj.value.lower(), maxiter=max_.value, Spinoff=spin.value)
        n.process_path = './Data'
        n.Data_file = os.path.join(n.process_path, file.value)
        n.run()
        # n.update()
        # clear_output()
        n.draw()
#         print(n.df.head())
#         source = ColumnDataSource(n.df)
#         p = figure(plot_width=900, plot_height=600)
#         p.line(x='Date', y='Qsim',source=source,line_color="red", line_width=2,legend_label='_filled',alpha = 0.8)
#         show(p)
#         print('run')        
button.on_click(on_button_clicked)


Button(description='Run', style=ButtonStyle())

Output()